In [1]:
import sagemaker
from sagemaker.pytorch import PyTorchModel

role = sagemaker.get_execution_role()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/24/25 14:22:21] INFO     Found credentials from IAM Role:                                   ]8;id=857322;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=7904;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[03/24/25 14:22:22] INFO     Found credentials from IAM Role:                                   ]8;id=799079;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=922496;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [24]:
# Define the PyTorch model (using local storage)
model_data = "deployment/model_final_2.tar.gz"
pytorch_model = PyTorchModel(
    model_data=model_data,
    role=role,
    entry_point="deployment/serve.py",
    framework_version="2.2",
    py_version="py310"
)

In [25]:
predictor = pytorch_model.deploy(instance_type="ml.c5.xlarge", initial_instance_count=1,container_startup_health_check_timeout=300,endpoint_name='malware-detection-v2-final',inference_response_timeout=900)

[03/24/25 15:01:15] INFO     Repacking model artifact (deployment/model_final_2.tar.gz), script        ]8;id=885746;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=689509;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/model.py#821\821]8;;\
                             artifact (None), and dependencies ([]) into single tar.gz file located at             
                             s3://sagemaker-us-east-1-548693114540/pytorch-inference-2025-03-24-15-01-             
                             14-842/model.tar.gz. This may take some time depending on model size...               

                    INFO     Creating model with name: pytorch-inference-2025-03-24-15-01-15-344    ]8;id=758812;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=621791;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[03/24/25 15:01:16] INFO     Creating endpoint-config with name malware-detection-v2-final          ]8;id=113219;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=928035;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#5937\5937]8;;\

                    INFO     Creating endpoint with name malware-detection-v2-final                 ]8;id=508878;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=499198;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4759\4759]8;;\

---------!

In [26]:
import numpy as np
def generate_random_array(num_samples=10, num_features=2381):
    return np.random.randn(num_samples, 1, num_features).astype(np.float32)

# Example usage
random_array = generate_random_array()
random_array[0][0]
# input_tensor_pred = torch.tensor(random_array,dtype = torch.float32)

array([0.42479023, 0.34560597, 0.4332203 , ..., 0.5255985 , 0.5559298 ,
       0.6815467 ], dtype=float32)

In [27]:
predictor.predict(random_array)

array([[1.00000000e+00],
       [1.04470611e-01],
       [1.46822401e-06],
       [1.78863183e-01],
       [6.94080293e-01],
       [9.99999881e-01],
       [9.99981523e-01],
       [6.35279536e-01],
       [8.88729274e-01],
       [2.04522982e-01]])

In [23]:
import boto3

endpoint_name = 'malware-detection-v2-final'
endpoint_config_name = 'malware-detection-v2-final'

sm_client = boto3.client('sagemaker')

# Fetch endpoint configuration name
endpoint_info = sm_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_config_name = endpoint_info['EndpointConfigName']

# # Delete the endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)
print(f"Deleted endpoint: {endpoint_name}")

# Delete the endpoint configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
print(f"Deleted endpoint configuration: {endpoint_config_name}")


Deleted endpoint: malware-detection-v2-final
Deleted endpoint configuration: malware-detection-v2-final
